# This is a basic Starter Kernel for the New York City Taxi Fare Prediction Playground Competition 
Here we'll use a simple linear model based on the travel vector from the taxi's pickup location to dropoff location which predicts the `fare_amount` of each ride.

This kernel uses some `pandas` and mostly `numpy` for the critical work.  There are many higher-level libraries you could use instead, for example `sklearn` or `statsmodels`.  

In [ ]:
# Initial Python environment setup...
import numpy as np # linear algebra
import pandas as pd # CSV file I/O (e.g. pd.read_csv)
import os # reading the input files we have access to

print(os.listdir('../input'))

### Setup training data
First let's read in our training data.  Kernels do not yet support enough memory to load the whole dataset at once, at least using `pd.read_csv`.  The entire dataset is about 55M rows, so we're skipping a good portion of the data, but it's certainly possible to build a model using all the data.

In [ ]:
types = {'fare_amount': 'float32',
         'pickup_longitude': 'float32',
         'pickup_latitude': 'float32',
         'dropoff_longitude': 'float32',
         'dropoff_latitude': 'float32',
         'passenger_count': 'uint8'}

# Columns to keep (basically discarding the 'key' column) - thanks to the suggestion by mhviraf
cols = ['fare_amount', 'pickup_datetime', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count']
train_df =  pd.read_csv('../input/train.csv',  parse_dates=["pickup_datetime"], usecols=cols, infer_datetime_format=True, dtype=types, nrows = 10_000_000)
train_df.dtypes

Let's create two new features in our training set representing the "travel vector" between the start and end points of the taxi ride, in both longitude and latitude coordinates.  We'll take the absolute value since we're only interested in distance traveled. Use a helper function since we'll want to do the same thing for the test set later.

In [ ]:
# Given a dataframe, add two new features 'abs_diff_longitude' and
# 'abs_diff_latitude' reprensenting the "Manhattan vector" from
# the pickup location to the dropoff location.
def add_travel_vector_features(df):
    df['abs_diff_longitude'] = (df.dropoff_longitude - df.pickup_longitude).abs()
    df['abs_diff_latitude'] = (df.dropoff_latitude - df.pickup_latitude).abs()

add_travel_vector_features(train_df)
del train_df['pickup_longitude']
del train_df['pickup_latitude']
del train_df['dropoff_longitude']
del train_df['dropoff_latitude']
train_df.dtypes
train_df.head(5)

### Explore and prune outliers
First let's see if there are any `NaN`s in the dataset.

In [ ]:
print(train_df.isnull().sum())

There are a small amount, so let's remove them from the dataset.

In [ ]:
print('Old size: %d' % len(train_df))
train_df = train_df.dropna(how = 'any', axis = 'rows')
print('New size: %d' % len(train_df))

Now let's quickly plot a subset of our travel vector features to see its distribution.

In [ ]:
#plot = train_df.iloc[:2000].plot.scatter('abs_diff_longitude', 'abs_diff_latitude')

We expect most of these values to be very small (likely between 0 and 1) since it should all be differences between GPS coordinates within one city.  For reference, one degree of latitude is about 69 miles.  However, we can see the dataset has extreme values which do not make sense.  Let's remove those values from our training set. Based on the scatterplot, it looks like we can safely exclude values above 5 (though remember the scatterplot is only showing the first 2000 rows...)

In [ ]:
print('Old size: %d' % len(train_df))
train_df = train_df[(train_df.abs_diff_longitude < 5.0) & (train_df.abs_diff_latitude < 5.0)]
print('New size: %d' % len(train_df))

In [ ]:
def add_hour_features(df):
    df['time_hour'] = df.pickup_datetime.apply(lambda x: x.hour)

add_hour_features(train_df)
del train_df['pickup_datetime']
print(train_df.dtypes)
train_df.head(5)

### Train our model
Our model will take the form $X \cdot w = y$ where $X$ is a matrix of input features, and $y$ is a column of the target variable, `fare_amount`, for each row. The weight column $w$ is what we will "learn".

First let's setup our input matrix $X$ and target column $y$ from our training set.  The matrix $X$ should consist of the two GPS coordinate differences, plus a third term of 1 to allow the model to learn a constant bias term.  The column $y$ should consist of the target `fare_amount` values.

In [ ]:
# Construct and return an Nx3 input matrix for our linear model
# using the travel vector, plus a 1.0 for a constant bias term.
def get_input_matrix(df):
    return np.column_stack((df.abs_diff_longitude, df.abs_diff_latitude,df.passenger_count, df.time_hour, np.ones(len(df))))
    #return np.column_stack((df.abs_diff_longitude, df.abs_diff_latitude, df.passenger_count, np.ones(len(df))))
    #return np.column_stack((df.pickup_longitude, df.pickup_latitude, df.dropoff_longitude, df.dropoff_latitude, df.passenger_count, np.ones(len(df))))

train_X = get_input_matrix(train_df)
train_y = np.array(train_df['fare_amount'])

print(train_X.shape)
print(train_y.shape)

In [ ]:
del train_df
import gc
gc.collect()

Now let's use `numpy`'s `lstsq` library function to find the optimal weight column $w$.

In [ ]:
# The lstsq function returns several things, and we only care about the actual weight vector w.
##(w, _, _, _) = np.linalg.lstsq(train_X, train_y, rcond = None)
##print(w)
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
model = Pipeline([('poly', PolynomialFeatures(degree=3)),('linear', LinearRegression(fit_intercept=False))])
model = model.fit(train_X,train_y)
model.named_steps['linear'].coef_

In [ ]:
del train_X
del train_y
import gc
gc.collect()

These weights pass a quick sanity check, since we'd expect the first two values -- the weights for the absolute longitude and latitude differences -- to be positive, as more distance should imply a higher fare, and we'd expect the bias term to loosely represent the cost of a very short ride.

Sidenote:  we can actually calculate the weight column $w$ directly using the [Ordinary Least Squares](https://en.wikipedia.org/wiki/Ordinary_least_squares) method:
$w = (X^T \cdot X)^{-1} \cdot X^T \cdot y$

### Make predictions on the test set
Now let's load up our test inputs and predict the `fare_amount`s for them using our learned weights!

In [ ]:
test_df = pd.read_csv('../input/test.csv',parse_dates=["pickup_datetime"], infer_datetime_format=True)
test_df.dtypes

In [ ]:
# Reuse the above helper functions to add our features and generate the input matrix.
add_travel_vector_features(test_df)
add_hour_features(test_df)

test_X = get_input_matrix(test_df)
# Predict fare_amount on the test set using our model (w) trained on the training set.
#test_y_predictions = np.matmul(test_X, w).round(decimals = 2)
test_y_predictions = np.array(model.predict(test_X))

# Write the predictions to a CSV file which we can submit to the competition.
submission = pd.DataFrame(
    {'key': test_df.key, 'fare_amount': test_y_predictions},
    columns = ['key', 'fare_amount'])
submission.to_csv('submission26_deg3.csv', index = False)

print(os.listdir('.'))

## Ideas for Improvement
The output here will score an RMSE of $5.74, but you can do better than that!  Here are some suggestions:

* Use more columns from the input data.  Here we're only using the start/end GPS points from columns `[pickup|dropoff]_[latitude|longitude]`.  Try to see if the other columns -- `pickup_datetime` and `passenger_count` -- can help improve your results.
* Use absolute location data rather than relative.  Here we're only looking at the difference between the start and end points, but maybe the actual values -- indicating where in NYC the taxi is traveling -- would be useful.
* Use a non-linear model to capture more intricacies within the data.
* Try to find more outliers to prune, or construct useful feature crosses.
* Use the entire dataset -- here we're only using about 20% of the training data!

Special thanks to Dan Becker, Will Cukierski, and Julia Elliot for reviewing this Kernel and providing suggestions!